# Congruence via Graph Distance

In this notebook, we try to calculate candidate node similarity via a simpler computational method than N-Degree Relations. This is also where we try to explore the potential of utilizing knowledge graphs. We take a list of candidate entities, represented as Item IDs, and search for that Item ID in either `source_item_id` or `target_item_id` in `statements_df`. This guarantees one layer of surrounding graph from the candidates, the potential for overlap and lets us measure candidate node importance via "Density" (number of first degree connections) or "Connectivity" (number of dendritic connections between candidate nodes).

#### Import Packages

In [1]:
import os
import time
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Progress bar
from tqdm import tqdm

## I. Load Data
### Integrate Predictions

In [2]:
# Base path to input
preds_path = '../../predictions/'

#instead of using wikipedia2vec sim candidate pools, use anchor link
entity_disambiguation2 = pd.read_csv(os.path.join(preds_path, "anchortext_frequency.csv"), delimiter=",")
entity_disambiguation2.head(10)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,norm_full_mention,mention_candidate_pools_page_ids,mention_candidate_pools_item_ids,candidate_pools_titles
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu,"[9317, 9239, 21347120, 9477, 1882861, 3261189,...","[458, 46, 211593, 1396, 363404, 3327447, 40537...","['European_Union', 'Europe', 'Eu,_Seine-Mariti..."
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german,"[11867, 11884, 152735, 21212, 12674, 290327, 1...","[183, 188, 42884, 7318, 43287, 141817, 181287,...","['Germany', 'German_language', 'Germans', 'Naz..."
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british,"[31717, 19097669, 13530298, 4721, 158019, 1522...","[145, 842438, 23666, 8680, 161885, 174193, 354...","['United_Kingdom', 'British_people', 'Great_Br..."
3,B,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","['Peter_Blackburn_(badminton)', 'Peter_Blackbu..."
4,I,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","['Peter_Blackburn_(badminton)', 'Peter_Blackbu..."
5,B,BRUSSELS,http://en.wikipedia.org/wiki/Brussels,3708.0,Brussels,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",brussels,"[3708, 575501, 1437181, 269753, 4152470, 23283...","[240, 239, 1050331, 28934, 800587, 994375, 142...","['Brussels', 'City_of_Brussels', 'R.W.D.M._Bru..."
6,B,European Commission,http://en.wikipedia.org/wiki/European_Commission,9974.0,European Commission,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",european commission,"[9974, 9974, 24468, 1130631, 1549462, 656283, ...","[8880, 8880, 8882, 388354, 1780232, 2661677, 8...","['European_Commission', 'European_Commission',..."
7,I,European Commission,http://en.wikipedia.org/wiki/European_Commission,9974.0,European Commission,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",european commission,"[9974, 9974, 24468, 1130631, 1549462, 656283, ...","[8880, 8880, 8882, 388354, 1780232, 2661677, 8...","['European_Commission', 'European_Commission',..."
8,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",german,"[11867, 11884, 152735, 21212, 12674, 290327, 1...","[183, 188, 42884, 7318, 43287, 141817, 181287,...","['Germany', 'German_language', 'Germans', 'Naz..."
9,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",british,"[31717, 19097669, 13530298, 4721, 158019, 1522...","[145, 842438, 23666, 8680, 161885, 174193, 354...","['United_Kingdom', 'British_people', 'Great_Br..."


In [3]:
# Confirm length
len(entity_disambiguation2)

29312

In [4]:
#since it's a bit too large, we'll use the same as in the smaller dataset: first 1000 rows
entity_disambiguation_new = entity_disambiguation2[:1000].copy()
print(len(entity_disambiguation_new))

1000


### Parse Saved Candidate Pool

Candidate pools when exported are typically stored as the string of a list. The below function parses the string back into a list with proper formatted values.

In [5]:
# Example
entity_disambiguation_new['candidate_pools_titles'][0][2:-2].split("', '")

['European_Union',
 'Europe',
 'Eu,_Seine-Maritime',
 'Europium',
 'Citizenship_of_the_European_Union',
 'United_Left_(Galicia)',
 'EU_(group)',
 'European_Union_law',
 'Eu_station',
 'Entropy']

In [6]:
# Function to parse list as string
def parse_list_string(list_string, value_type=int):
    
    parsed_list = []
    
    # If candidate pool is empty
    if list_string == "[]" or isinstance(list_string, float):
        pass
    # Else parse
    else:
        # Parses lists of titles as strings
        if value_type==str:
            # Eliminate bracket and parenthesis on either side, split by comma pattern
            parsed_list = re.split("', '|\", \"|', \"|\", \'", list_string[2:-2])

        # Parses lists of IDs as ints
        elif value_type==int:
            # Eliminate brackets and convert each number from string to int
            parsed_list = list(map(int, list_string[1:-1].split(', ')))
            
        
    return parsed_list


In [7]:
# Manually test function
# 0 is the hard one. See how some value is stored with '' and some with "". Unsure why.
parse_list_string(entity_disambiguation_new['candidate_pools_titles'][0], value_type=str)

['European_Union',
 'Europe',
 'Eu,_Seine-Maritime',
 'Europium',
 'Citizenship_of_the_European_Union',
 'United_Left_(Galicia)',
 'EU_(group)',
 'European_Union_law',
 'Eu_station',
 'Entropy']

In [8]:
# Apply defined function
parsed_candidate_pool = entity_disambiguation_new['candidate_pools_titles'].apply(parse_list_string, value_type=str)
entity_disambiguation_new['candidate_pools_titles'] = parsed_candidate_pool
entity_disambiguation_new['candidate_pools_titles'][:3]

0    [European_Union, Europe, Eu,_Seine-Maritime, E...
1    [Germany, German_language, Germans, Nazi_Germa...
2    [United_Kingdom, British_people, Great_Britain...
Name: candidate_pools_titles, dtype: object

In [9]:
# Apply defined function
parsed_candidate_pool = entity_disambiguation_new['mention_candidate_pools_item_ids'].apply(parse_list_string, value_type=int)
entity_disambiguation_new['mention_candidate_pools_item_ids'] = parsed_candidate_pool
entity_disambiguation_new['mention_candidate_pools_item_ids'][:3]

0    [458, 46, 211593, 1396, 363404, 3327447, 40537...
1    [183, 188, 42884, 7318, 43287, 141817, 181287,...
2    [145, 842438, 23666, 8680, 161885, 174193, 354...
Name: mention_candidate_pools_item_ids, dtype: object

In [10]:
# Apply defined function
parsed_candidate_pool = entity_disambiguation_new['mention_candidate_pools_page_ids'].apply(parse_list_string, value_type=int)
entity_disambiguation_new['mention_candidate_pools_page_ids'] = parsed_candidate_pool
entity_disambiguation_new['mention_candidate_pools_page_ids'][:3]

0    [9317, 9239, 21347120, 9477, 1882861, 3261189,...
1    [11867, 11884, 152735, 21212, 12674, 290327, 1...
2    [31717, 19097669, 13530298, 4721, 158019, 1522...
Name: mention_candidate_pools_page_ids, dtype: object

In [11]:
# Integrate page.csv
page_df = pd.read_csv("../../data/kdkg/page.csv", delimiter=',')
assert type(page_df) is not None
display(page_df.head(3))

,page_id,item_id,title,views
0,12,6199,Anarchism,31335
1,25,38404,Autism,49693
2,39,101038,Albedo,14573


In [12]:
# Test manual search
page_df[page_df['title'] == "Russians in the United Kingdom"]

,page_id,item_id,title,views
1352883,9808786,3308075,Russians in the United Kingdom,684


In [13]:
entity_disambiguation_new.head()

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,norm_full_mention,mention_candidate_pools_page_ids,mention_candidate_pools_item_ids,candidate_pools_titles
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu,"[9317, 9239, 21347120, 9477, 1882861, 3261189,...","[458, 46, 211593, 1396, 363404, 3327447, 40537...","[European_Union, Europe, Eu,_Seine-Maritime, E..."
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german,"[11867, 11884, 152735, 21212, 12674, 290327, 1...","[183, 188, 42884, 7318, 43287, 141817, 181287,...","[Germany, German_language, Germans, Nazi_Germa..."
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british,"[31717, 19097669, 13530298, 4721, 158019, 1522...","[145, 842438, 23666, 8680, 161885, 174193, 354...","[United_Kingdom, British_people, Great_Britain..."
3,B,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","[Peter_Blackburn_(badminton), Peter_Blackburn_..."
4,I,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","[Peter_Blackburn_(badminton), Peter_Blackburn_..."


In [14]:
#Append congruent mention ids
#Here we take 'anchor link most viewed' as 'ground truth' labels

congruent_mention_ids_anchor_link = []
for i in range(len(entity_disambiguation_new)):
    full_mention = entity_disambiguation_new.full_mention[i]
    sentence_id = entity_disambiguation_new.sentence_id[i]
    new_df = entity_disambiguation_new[entity_disambiguation_new.sentence_id==sentence_id]
    new_df = new_df[new_df.full_mention!=full_mention]
    ids = new_df.mention_candidate_pools_item_ids.values
    ids_final = []
    for j in ids:
        try:
            ids_final.append(j[0])
        except:
            pass
    congruent_mention_ids_anchor_link.append(ids_final)

In [15]:
#Append the produced congruent mention ids
entity_disambiguation_new['congruent_mention_ids_anchor_link'] = congruent_mention_ids_anchor_link
entity_disambiguation_new.head()

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,norm_full_mention,mention_candidate_pools_page_ids,mention_candidate_pools_item_ids,candidate_pools_titles,congruent_mention_ids_anchor_link
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu,"[9317, 9239, 21347120, 9477, 1882861, 3261189,...","[458, 46, 211593, 1396, 363404, 3327447, 40537...","[European_Union, Europe, Eu,_Seine-Maritime, E...","[183, 145]"
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german,"[11867, 11884, 152735, 21212, 12674, 290327, 1...","[183, 188, 42884, 7318, 43287, 141817, 181287,...","[Germany, German_language, Germans, Nazi_Germa...","[458, 145]"
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british,"[31717, 19097669, 13530298, 4721, 158019, 1522...","[145, 842438, 23666, 8680, 161885, 174193, 354...","[United_Kingdom, British_people, Great_Britain...","[458, 183]"
3,B,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","[Peter_Blackburn_(badminton), Peter_Blackburn_...","[240, 8880, 8880, 183, 145]"
4,I,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,"[56783206, 9643132, 56873217]","[2073954, 7172840, 26634508]","[Peter_Blackburn_(badminton), Peter_Blackburn_...","[240, 8880, 8880, 183, 145]"


In [16]:
#how many true ids are actually in candidate ids
true = entity_disambiguation_new.wikipedia_page_ID
notnull = np.where(~np.isnan(true))[0]
true = [true[i] for i in notnull]
can = entity_disambiguation_new.mention_candidate_pools_page_ids.values
can = [can[i] for i in notnull]
in_or_not = [true[i] in can[i] for i in range(len(true))]

print('{}% of true ids are in candidate ids'.format(sum(in_or_not)/len(true)*100))

94.96402877697841% of true ids are in candidate ids


In [17]:
# Integrate statements.csv
#This is our knowledge graph
statements_df = pd.read_csv("../../data/kdkg/statements.csv", delimiter=',')
assert type(statements_df) is not None
display(statements_df.head(3))

,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745


In [18]:
len(statements_df)

141206853

In [19]:
# How many sentences exist in our data
sentence_count = max(entity_disambiguation_new['sentence_id'])
print("{:,} sentences in our data.".format(sentence_count))

221 sentences in our data.


## II. Modeling

### 2.1 Shortest Path Approach

Generate Graph of Dendrites

From this pool of candidate IDs, we are going to generate our one-degree in both direction dendritic graph from `statements.csv`.

In [41]:
%%time
# Manually test on one candidate pool
rand_idx = 1
print("Generating Values for {}: {}".format(rand_idx, entity_disambiguation_new['full_mention'][rand_idx]))
print('Testing dataframe search time')
candidate_pool = entity_disambiguation_new['mention_candidate_pools_item_ids'][rand_idx] + entity_disambiguation_new['congruent_mention_ids_anchor_link'][rand_idx]
candidate_pool_dendrites = statements_df[statements_df['source_item_id'].isin(candidate_pool)\
             | statements_df['target_item_id'].isin(candidate_pool)]
display(candidate_pool_dendrites)

Generating Values for 1: German
Testing dataframe search time


,source_item_id,edge_property_id,target_item_id
371,16,530,145
375,16,530,183
397,16,530,458
433,16,807,145
434,16,807,145
...,...,...,...
141201967,77254864,1412,188
141202910,77255368,17,183
141205122,77256485,921,183
141205152,77256499,27,145


CPU times: user 4.15 s, sys: 5.42 s, total: 9.57 s
Wall time: 10.7 s


#### Create Dendritic Graph using NetworkX

In [22]:
# import NetworkX
import networkx as nx

In [43]:
%%time
# Create NX graph from Pandas dataframe
print('testing networkx object creation time')
G = nx.from_pandas_edgelist(candidate_pool_dendrites,
                           source='source_item_id', target='target_item_id',
                           edge_attr=True, create_using=nx.MultiGraph())

testing networkx object creation time
CPU times: user 18.4 s, sys: 4.03 s, total: 22.4 s
Wall time: 23 s


In [44]:
# Print number of nodes in graph
print("Number of nodes: ", G.number_of_nodes())
print("Number of links: ", G.number_of_edges())

Number of nodes:  2000602
Number of links:  2187009


#### Create Distance Matrix

By taking every candidate and computing the distance to every other candidate, we can create a distance matrix showing which nodes (if any) are closer to each other. We will then select the most connected or closest connected node from the candidate pool as the predicted entity.

In [45]:
h = len(entity_disambiguation_new['mention_candidate_pools_item_ids'][rand_idx])
w = len(entity_disambiguation_new['congruent_mention_ids_anchor_link'][rand_idx])
print("Matrix will be of size {}x{}".format(h, w))

Matrix will be of size 10x2


In [46]:
%%time
# Iteratively create matrix
shortest_path_matrix = []
for candidate_src in entity_disambiguation_new['mention_candidate_pools_item_ids'][rand_idx]:
    shortest_paths = []
#     print("Source: ", candidate_src)
    for candidate_trgt in entity_disambiguation_new['congruent_mention_ids_anchor_link'][rand_idx]:
#         print("Target: ", candidate_trgt)
        if candidate_src is None or candidate_trgt is None:
            shortest_paths.append(None)
        else:
            try:
                shortest_paths.append(int(nx.shortest_path_length(G, source=candidate_src, target=candidate_trgt)))
            except:
                shortest_paths.append(-1)
    shortest_path_matrix.append(shortest_paths)

CPU times: user 329 ms, sys: 36.3 ms, total: 365 ms
Wall time: 364 ms


In [47]:
# Display matrix with closest nodes highlighted
shortest_path_df = pd.DataFrame(shortest_path_matrix)
shortest_path_df.style.apply(lambda x: ["background: skyblue" if v == 1 else "" for v in x], axis = 1)

,0,1
0,1,1
1,1,2
2,2,2
3,2,2
4,2,2
5,2,2
6,3,2
7,2,2
8,3,3
9,3,2


In [48]:
# Count node with lowest average distance
rank_by_mean_path = np.mean(shortest_path_df, axis=1).sort_values(ascending=True)
display(rank_by_mean_path)
# Filter out -1 values representing no path
prediction_by_mean_rank = rank_by_mean_path[rank_by_mean_path >= 0].index[0]
print("Generating Values for {}: {}".format(rand_idx, entity_disambiguation_new['full_mention'][rand_idx]))
print("Prediction would be {}".format(page_df[page_df['item_id'] == candidate_pool[prediction_by_mean_rank]]['title'].values))

0    1.0
1    1.5
2    2.0
3    2.0
4    2.0
5    2.0
7    2.0
6    2.5
9    2.5
8    3.0
dtype: float64

Generating Values for 1: German
Prediction would be ['Germany']


#### Test running the model on a sentence
Here we try predicting a whole sentence instead of single mentions in order to save time

In [55]:
#candidate pool: all entities and candidates within this one sentence

sentence_id = 2
sentence_df = entity_disambiguation_new[entity_disambiguation_new.sentence_id==sentence_id]
#all candidates
t = list(sentence_df.mention_candidate_pools_item_ids)
candidates = [item for sublist in t for item in sublist]
#all mentions
t1 = list(sentence_df.congruent_mention_ids_anchor_link)
congruence = [item for sublist in t1 for item in sublist]
#the whole list we need from edges dataframe
candidate_pool = candidates + congruence
candidate_pool = list(set(candidate_pool))
print('{} entities in total in this sentence (including candidates)'.format(len(candidate_pool)))

29 entities in total in this sentence (including candidates)


In [56]:
%%time
print('Testing dataframe search time')
candidate_pool_dendrites = statements_df[statements_df['source_item_id'].isin(candidate_pool)\
             | statements_df['target_item_id'].isin(candidate_pool)]
display(candidate_pool_dendrites)

Testing dataframe search time


,source_item_id,edge_property_id,target_item_id
371,16,530,145
375,16,530,183
397,16,530,458
433,16,807,145
434,16,807,145
...,...,...,...
141201961,77254864,27,183
141202910,77255368,17,183
141205122,77256485,921,183
141205152,77256499,27,145


CPU times: user 9.05 s, sys: 7.81 s, total: 16.9 s
Wall time: 18.3 s


In [57]:
#networkx object

start = time.time()
G = nx.from_pandas_edgelist(candidate_pool_dendrites,
                       source='source_item_id', target='target_item_id',
                       edge_attr=True, create_using=nx.MultiGraph())
execution_time = (time.time()-start)/60
print('execution time: ', execution_time, ' mins')

execution time:  0.4146684686342875  mins


In [58]:
#create shortest path matrix
shortest_path_matrix = []
for candidate_src in candidates:
    shortest_paths = []
    for candidate_trgt in congruence:
        if candidate_src is None or candidate_trgt is None:
            shortest_paths.append(None)
        else:
            try:
                shortest_paths.append(int(nx.shortest_path_length(G, source=candidate_src, target=candidate_trgt)))
            except:
                shortest_paths.append(-1)
    shortest_path_matrix.append(shortest_paths)

In [59]:
#predict on each mention
pred = []
for i in range(len(sentence_df)):
    sub_candidates = sentence_df['mention_candidate_pools_item_ids'].values[i]
    sub_candidates_indices = [candidates.index(k) for k in sub_candidates]
    sub_congruent = sentence_df.congruent_mention_ids_anchor_link.values[i]
    sub_congruent_indices = [congruence.index(k) for k in sub_congruent]
    new_mtx = np.take(shortest_path_matrix, sub_candidates_indices, 0)
    new_mtx = np.take(new_mtx, sub_congruent_indices, 1)
    rank_by_mean_path = np.mean(new_mtx, axis=1).argsort() #lowest to highest
    # Filter out -1 values representing no path
    try:
        prediction_by_mean_rank = rank_by_mean_path[0]
    except:
        #if we didn't find any ranks, then take the first candidate (most similar by wikipedia2vec)
        prediction_by_mean_rank = 0
#     print(i, sub_candidates, prediction_by_mean_rank)
    try:
        pred.append(sub_candidates[prediction_by_mean_rank])
    except:
        pred.append(None)
pred

[183, 1376407, 1376407, None, None, 977566]

#### Predicting on each sentence and produce accuracy

In [20]:
#put it all in one function
def predict_sentence(sentence_id):
    sentence_df = entity_disambiguation_new[entity_disambiguation_new.sentence_id==sentence_id]
    #all candidates
    t = list(sentence_df.mention_candidate_pools_item_ids)
    candidates = [item for sublist in t for item in sublist]
    #all mentions
    t1 = list(sentence_df.congruent_mention_ids_anchor_link)
    congruence = [item for sublist in t1 for item in sublist]
    #the whole list we need from edges dataframe
    candidate_pool = candidates + congruence
    candidate_pool = list(set(candidate_pool))
    
    candidate_pool_dendrites = statements_df[statements_df['source_item_id'].isin(candidate_pool)\
         | statements_df['target_item_id'].isin(candidate_pool)]
    
    G = nx.from_pandas_edgelist(candidate_pool_dendrites,
                       source='source_item_id', target='target_item_id',
                       edge_attr=True, create_using=nx.MultiGraph())
    
    #create shortest path matrix
    shortest_path_matrix = []
    for candidate_src in candidates:
        shortest_paths = []
        for candidate_trgt in congruence:
            if candidate_src is None or candidate_trgt is None:
                shortest_paths.append(None)
            else:
                try:
                    shortest_paths.append(int(nx.shortest_path_length(G, source=candidate_src, target=candidate_trgt)))
                except:
                    shortest_paths.append(-1)
        shortest_path_matrix.append(shortest_paths)
        
    #predict on each mention
    pred = []
    for i in range(len(sentence_df)):
        sub_candidates = sentence_df['mention_candidate_pools_item_ids'].values[i]
        sub_candidates_indices = [candidates.index(k) for k in sub_candidates]
        sub_congruent = sentence_df.congruent_mention_ids_anchor_link.values[i]
        sub_congruent_indices = [congruence.index(k) for k in sub_congruent]
        new_mtx = np.take(shortest_path_matrix, sub_candidates_indices, 0)
        new_mtx = np.take(new_mtx, sub_congruent_indices, 1)
        rank_by_mean_path = np.mean(new_mtx, axis=1).argsort() #lowest to highest
        # Filter out -1 values representing no path
        try:
            prediction_by_mean_rank = rank_by_mean_path[0]
        except:
            prediction_by_mean_rank = 0
        
        try:
            pred.append(sub_candidates[prediction_by_mean_rank])
        except:
            pred.append(None)
    
    return pred

In [24]:
#predict on our dataframe
#also save the networkx object for later modeling on embeddings
pred1 = []
for i in tqdm(entity_disambiguation_new.sentence_id.unique()[:50]):
    pred = predict_sentence(i)
    pred1.append(pred)



  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:41<34:14, 41.93s/it]

  4%|▍         | 2/50 [02:03<43:08, 53.93s/it]

  6%|▌         | 3/50 [02:51<40:41, 51.94s/it]/Users/esthe/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/esthe/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


  8%|▊         | 4/50 [02:58<29:28, 38.46s/it]/Users/esthe/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/esthe/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


 10%|█         | 5/50 [03:01<21:02, 28.05s/it]

 12%|█▏        | 6/50 [03:06<15:26, 21

In [69]:
#helper function that converts a list of item ids to mention texts
def item_to_mention(l):
    mentions = []
    for i in l:
        try:
            mentions.append(page_df[page_df.item_id==i].title.values[0])
        except:
            mentions.append(None)
    return mentions

In [31]:
#evaluate accuracy
#first flatten our list
pred_mention = [item for sublist in pred1 for item in sublist]
pred_mention = item_to_mention(pred_mention)
true = [str(i).lower() for i in entity_disambiguation_new['wikipedia_title'][:len(pred_mention)]]
#only filter not null values and test accuracies
notnull_new = [i for i in notnull if i<=len(true)]
true = [true[i] for i in notnull_new]

pred_mention = [str(i).lower() for i in pred_mention]
#only filter not null values and test accuracies
pred_mention = [pred_mention[i] for i in notnull_new]

accurate_predictions = sum([true[i]==pred_mention[i] for i in range(len(true))])
print("****************************")
print(f"Predictive Accuracy: {round(accurate_predictions / len(true) *100, 3)}%")
print("****************************")

****************************
Predictive Accuracy: 60.163%
****************************


### 2.2 Embeddings and Euclidean distance / cosine similarity

#### Example with one sentence

In [20]:
sentence_df = entity_disambiguation_new[entity_disambiguation_new.sentence_id == 2]
sentence_df.head(10)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,norm_full_mention,mention_candidate_pools_page_ids,mention_candidate_pools_item_ids,candidate_pools_titles,congruent_mention_ids_anchor_link
10,B,Germany,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,2,0,"['Germany', 'European Union', 'Werner Zwingman...",germany,"[11867, 250204, 21212, 12674, 33685, 662281, 1...","[183, 43310, 7318, 43287, 41304, 154408, 12031...","[Germany, Germany_national_football_team, Nazi...","[458, 458, 145]"
11,B,European Union,http://en.wikipedia.org/wiki/European_Union,9317.0,European Union,2,0,"['Germany', 'European Union', 'Werner Zwingman...",european union,"[9317, 1933156, 9317, 10890716, 276436, 265743...","[458, 1376407, 458, 185441, 208202, 319328, 36...","[European_Union, European_Boxing_Union, Europe...","[183, 145]"
12,I,European Union,http://en.wikipedia.org/wiki/European_Union,9317.0,European Union,2,0,"['Germany', 'European Union', 'Werner Zwingman...",european union,"[9317, 1933156, 9317, 10890716, 276436, 265743...","[458, 1376407, 458, 185441, 208202, 319328, 36...","[European_Union, European_Boxing_Union, Europe...","[183, 145]"
13,B,Werner Zwingmann,NaN,NaN,NaN,2,0,"['Germany', 'European Union', 'Werner Zwingman...",werner zwingmann,[],[],[],"[183, 458, 458, 145]"
14,I,Werner Zwingmann,NaN,NaN,NaN,2,0,"['Germany', 'European Union', 'Werner Zwingman...",werner zwingmann,[],[],[],"[183, 458, 458, 145]"
15,B,Britain,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,2,0,"['Germany', 'European Union', 'Werner Zwingman...",britain,"[31717, 13530298, 152256, 158019, 13525, 4721,...","[145, 23666, 174193, 161885, 185103, 8680, 977...","[United_Kingdom, Great_Britain, United_Kingdom...","[183, 458, 458]"


In [21]:
#load in pretrained embeddings by graphvite
#downloaded here:
#https://graphvite.io/docs/latest/pretrained_model.html
import pickle
with open("../../data/transe_wikidata5m.pkl", "rb") as fin:
    model = pickle.load(fin)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [22]:
#helper function from graphvite package
#https://github.com/DeepGraphLearning/graphvite/blob/master/python/graphvite/dataset.py
#would be easier to use the package directly, I am using the raw file here because of downloading issues
def load_alias(alias_file):
    alias2object = {}
    ambiguous = set()
    with open(alias_file, "r") as fin:
        for line in fin:
            tokens = line.strip().split("\t")
            object = tokens[0]
            for alias in tokens[1:]:
                if alias in alias2object and alias2object[alias] != object:
                    ambiguous.add(alias)
                alias2object[alias] = object
        for alias in ambiguous:
            alias2object.pop(alias)
    return alias2object

In [23]:
#load in the entity file, downloaded here: 
#https://www.dropbox.com/s/bgmgvk8brjwpc9w/entity.txt.gz?dl=1
my_file = "../../data/entity.txt"
entity = load_alias(my_file)

In [30]:
#for one entity, this is how we will find its embeddings:
e = 'germany'
ind = entity[e]
ind = entity2id[ind]
embedding = entity_embeddings[ind]
print('final embedding is {} dimensions'.format(len(embedding)))

#put this into a helper function
def ent2emb(ent):
    ind = entity[ent]
    ind = entity2id[ind]
    embedding = entity_embeddings[ind]
    return embedding

final embedding is 512 dimensions


In [32]:
#sanity check: germany - berlin is shorter distance to germany - michael jordan
close = 'berlin'
far = 'michael jordan'
dis_close = np.linalg.norm(ent2emb(e) - ent2emb(close))
dis_far = np.linalg.norm(ent2emb(e) - ent2emb(far))
print('distance between {} and {}: {}'.format(e, close, dis_close))
print('distance between {} and {}: {}'.format(e, far, dis_far))

distance between germany and berlin: 0.4656655788421631
distance between germany and michael jordan: 1.042931318283081


In [52]:
#euclidean distance

def ed(l1, l2):
    return np.linalg.norm(l1-l2)

from scipy import spatial
def cd(l1, l2):
    return spatial.distance.cosine(l1, l2)

Testing the algorithm for Euclidean Distance

In [58]:
#all candidates
t = list(sentence_df.candidate_pools_titles)
candidates = [item for sublist in t for item in sublist]
#all mentions
t1 = list(sentence_df.congruent_mentions)
congruence = [item for sublist in t1 for item in sublist]
#the whole list we need from edges dataframe
candidate_pool = candidates + congruence
#remove duplicates
candidate_pool = list(set(candidate_pool))

#create shortest path matrix
shortest_path_matrix = []
for candidate_src in candidates:
    shortest_paths = []
    for candidate_trgt in congruence:
        if candidate_src is None or candidate_trgt is None:
            shortest_paths.append(None)
        else:
            try:
                shortest_paths.append(ed(ent2emb(candidate_src), ent2emb(candidate_trgt)))
            except:
                shortest_paths.append(-1)
    shortest_path_matrix.append(shortest_paths)

#predict on each mention
pred = []
for i in range(len(sentence_df)):
    sub_candidates = sentence_df['candidate_pools_titles'].values[i]
    sub_candidates_indices = [candidates.index(k) for k in sub_candidates]
    sub_congruent = sentence_df.congruent_mentions.values[i]
    sub_congruent_indices = [congruence.index(k) for k in sub_congruent]
    new_mtx = np.take(shortest_path_matrix, sub_candidates_indices, 0)
    new_mtx = np.take(new_mtx, sub_congruent_indices, 1)
    rank_by_mean_path = np.mean(new_mtx, axis=1).argsort() #lowest to highest
    # Filter out -1 values representing no path
    try:
        prediction_by_mean_rank = rank_by_mean_path[0]
    except:
        prediction_by_mean_rank = 0

    try:
        pred.append(sub_candidates[prediction_by_mean_rank])
    except:
        pred.append(None)

In [59]:
pred

['Germany', 'European_Union', 'European_Union', None, None, 'United_Kingdom']

Testing the algorithm for Cosine Distance

In [61]:
#all candidates
t = list(sentence_df.candidate_pools_titles)
candidates = [item for sublist in t for item in sublist]
#all mentions
t1 = list(sentence_df.congruent_mentions)
congruence = [item for sublist in t1 for item in sublist]
#the whole list we need from edges dataframe
candidate_pool = candidates + congruence
#remove duplicates
candidate_pool = list(set(candidate_pool))

#create shortest path matrix
shortest_path_matrix = []
for candidate_src in candidates:
    shortest_paths = []
    for candidate_trgt in congruence:
        if candidate_src is None or candidate_trgt is None:
            shortest_paths.append(None)
        else:
            try:
                shortest_paths.append(cd(ent2emb(candidate_src), ent2emb(candidate_trgt)))
            except:
                shortest_paths.append(-1)
    shortest_path_matrix.append(shortest_paths)

#predict on each mention
pred = []
for i in range(len(sentence_df)):
    sub_candidates = sentence_df['candidate_pools_titles'].values[i]
    sub_candidates_indices = [candidates.index(k) for k in sub_candidates]
    sub_congruent = sentence_df.congruent_mentions.values[i]
    sub_congruent_indices = [congruence.index(k) for k in sub_congruent]
    new_mtx = np.take(shortest_path_matrix, sub_candidates_indices, 0)
    new_mtx = np.take(new_mtx, sub_congruent_indices, 1)
    rank_by_mean_path = np.mean(new_mtx, axis=1).argsort() #lowest to highest
    # Filter out -1 values representing no path
    try:
        prediction_by_mean_rank = rank_by_mean_path[0]
    except:
        prediction_by_mean_rank = 0

    try:
        pred.append(sub_candidates[prediction_by_mean_rank])
    except:
        pred.append(None)

In [62]:
pred

['Germany', 'European_Union', 'European_Union', None, None, 'United_Kingdom']

#### Repeat this and produce accuracy scores

In [24]:
def predict_sentence_emb(sentence_id, method):
    sentence_df = entity_disambiguation_new[entity_disambiguation_new.sentence_id==sentence_id]
    #all candidates
    t = list(sentence_df.candidate_pools_titles)
    candidates = [item for sublist in t for item in sublist]
    #all mentions
    t1 = list(sentence_df.congruent_mentions)
    congruence = [item for sublist in t1 for item in sublist]
    #the whole list we need from edges dataframe
    candidate_pool = candidates + congruence
    #remove duplicates
    candidate_pool = list(set(candidate_pool))

    #create shortest path matrix
    shortest_path_matrix = []
    for candidate_src in candidates:
        shortest_paths = []
        for candidate_trgt in congruence:
            if candidate_src is None or candidate_trgt is None:
                shortest_paths.append(None)
            else:
                try:
                    if method=='euclidean distance':
                        shortest_paths.append(ed(ent2emb(candidate_src), ent2emb(candidate_trgt)))
                    if method=='cosine distance':
                        shortest_paths.append(ed(ent2emb(candidate_src), ent2emb(candidate_trgt)))
                except:
                    shortest_paths.append(-1)
        shortest_path_matrix.append(shortest_paths)

    #predict on each mention
    pred = []
    for i in range(len(sentence_df)):
        try:
            sub_candidates = sentence_df['candidate_pools_titles'].values[i]
            sub_candidates_indices = [candidates.index(k) for k in sub_candidates]
            sub_congruent = sentence_df.congruent_mentions.values[i]
            sub_congruent_indices = [congruence.index(k) for k in sub_congruent]
            new_mtx = np.take(shortest_path_matrix, sub_candidates_indices, 0)
            new_mtx = np.take(new_mtx, sub_congruent_indices, 1)
            rank_by_mean_path = np.mean(new_mtx, axis=1).argsort() #lowest to highest
            # Filter out -1 values representing no path
            prediction_by_mean_rank = rank_by_mean_path[0]
        except:
            prediction_by_mean_rank = 0

        try:
            pred.append(sub_candidates[prediction_by_mean_rank])
        except:
            pred.append(None)
    return pred

In [25]:
#EUCLIDEAN DISTANCE
#predict on our dataframe
pred_ed = []
for i in tqdm(entity_disambiguation_new.sentence_id.unique()[:100]):
    pred = predict_sentence_emb(i, 'euclidean distance')
    pred_ed.append(pred)

100%|██████████| 100/100 [00:02<00:00, 44.63it/s]


In [27]:
#evaluate accuracy
#first flatten our list
pred_mention = [item for sublist in pred_ed for item in sublist]
true = [str(i).lower() for i in entity_disambiguation_new['wikipedia_title'][:len(pred_mention)]]
#only filter not null values and test accuracies
notnull_new = [i for i in notnull if i<=len(true)]
true = [true[i] for i in notnull_new]

pred_mention = [str(i).lower().replace('_', ' ') for i in pred_mention]
#only filter not null values and test accuracies
pred_mention = [pred_mention[i] for i in notnull_new]
print('EUCLIDEAN DISTANCE:')
print('predicted {} entities'.format(len(true)))
accurate_predictions = sum([true[i]==pred_mention[i] for i in range(len(true))])
print("****************************")
print(f"Predictive Accuracy: {round(accurate_predictions / len(true) *100, 3)}%")
print("****************************")

EUCLIDEAN DISTANCE:
predicted 318 entities
****************************
Predictive Accuracy: 74.528%
****************************


In [28]:
#COSINE DISTANCE
#predict on our dataframe
pred_cd = []
for i in tqdm(entity_disambiguation_new.sentence_id.unique()[:100]):
    pred = predict_sentence_emb(i, 'cosine distance')
    pred_cd.append(pred)

100%|██████████| 100/100 [00:02<00:00, 40.23it/s]


In [29]:
#evaluate accuracy
#first flatten our list
pred_mention = [item for sublist in pred_cd for item in sublist]
true = [str(i).lower() for i in entity_disambiguation_new['wikipedia_title'][:len(pred_mention)]]
#only filter not null values and test accuracies
notnull_new = [i for i in notnull if i<=len(true)]
true = [true[i] for i in notnull_new]

pred_mention = [str(i).lower().replace('_', ' ') for i in pred_mention]
#only filter not null values and test accuracies
pred_mention = [pred_mention[i] for i in notnull_new]
print('EUCLIDEAN DISTANCE:')
accurate_predictions = sum([true[i]==pred_mention[i] for i in range(len(true))])
print("****************************")
print(f"Predictive Accuracy: {round(accurate_predictions / len(true) *100, 3)}%")
print("****************************")

EUCLIDEAN DISTANCE:
****************************
Predictive Accuracy: 74.528%
****************************
